In [4]:
import sys
sys.path.append('..')
import crystalmaths
#Interactive mode necessary to select planes in image.
%matplotlib qt

# Create ImageHandler

Provide a filepath as the input to the ImageHandler class. Optional argument is window_type. 
Please read about the implications of applying a window to an image, and alternative options. 
The purpose of applying the window is to decrease the contribution of features near the outside of the image, e.g. the scalebar. 


## Q. Why did cropping the scale bar out not remove the streak? 


A: Discontinuities at border being interpreted as features by FFT.

Resources:
https://dsp.stackexchange.com/questions/25927/fourier-transform-artifacts 
https://www.cambridge.org/core/journals/microscopy-and-microanalysis/article/periodic-artifact-reduction-in-fourier-transforms-of-full-field-atomic-resolution-images/80D0E226F0B4B16627AA0B6B9BD24F24

see demo: https://github.com/jacobkimmel/ps_decomp

Also consider:
https://dsp.stackexchange.com/questions/26830/python-scipy-fft-on-numpy-hanning-window-smears-peaks


In [5]:
image_filepath = r'../Local_data/SL_Bornite_011.png'
image_object = crystalmaths.imagetools.ImageHandler(image_filepath=image_filepath, window_type = 'hanning')
image_object.show_image()

# Interacting with ImageHandler object called image_object

image_object is an object which contains useful attributes used later to perform a search of a crystal structure database.

1. Input the pixel:nm scale ratio for the image. This can be done by either inputting it as an argument to the .get_scale_ratio method, or by keeping the method call blank. The latter opens your image to allow scalebar selection by clicking on the image.
2. Next, we want to select two planes in the image to index. Use the .get_planes() method. This will open up your TEM image and prompt you to click on the planes you are interested in performing a search on. 

In [6]:
image_object.get_scale_ratio()

Enter scalebar length in nanometers.2
length_nm =  2.0
length_pixel = 20.12987012987014


In [7]:
image_object.get_planes()

Pick two planes using left mouse button. Right click once done. Middle mouse button removes most recent point.
Saved points =  [(304.474025974026, 165.0681818181818), (252.47186147186147, 30.030303030303003)]
Press keyboard button to save points, mouseclick to restart


### Use image_object to Perform Search

arguments for compile_results are: 
- image_object: which is generated above in this notebook 
- d_spacing_tolerance: which is how strict the querry is to the American Minerals Society Database (we reccomend 0.2 because of uncertainty from clicking a pixel in the user interface)
- angle_tolerance: which is how strict the analysis is for returning pairs of planes with matching angles between them (we reccomend 10 degrees because of uncertainty from clicking a pixel in the user interface) 

result_list = [(pandas dataframe1, pandas dataframe2), ...]
This is a list of tuples where each item is a result from the American Minearologist Crystal Structure Database
- First dataframe contains analysis results, which shows what the possible zone axis and mineral name are
- Second dataframe contains other metadata obtained during the search

In [8]:
result_list = crystalmaths.compile_results.compile_results(
    image_object=image_object, d_spacing_tolerance=0.2, angle_tolerance=10, mineral='Bornite')

Searching this page: http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(2.061503918298683,1.9563936966129438),opt(),type(d-spacing),tolerance(0.2)&mineral=Bornite
No results identified from search.


In [9]:
for result in result_list:
    print(result)